In [1]:
import fasttext
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 1000)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import gensim

In [28]:
df = pd.read_csv('gold_lower.csv')
df = df.sample(frac=1).reset_index(drop=True)
df['target'].value_counts()

0.0    462
1.0    356
Name: target, dtype: int64

In [29]:
df[df['target']==1].head()

,text,target
1,жінко твій день восьме березня,1.0
2,жінка має народжувати а чоловік забезпечувати,1.0
3,навіщо ти переймаєшся за свою кар'єру ти повина сидіти з дітьми вдома,1.0
5,емоції це про жінок а логіка про чоловіків,1.0
6,чоловік повинен заробляти більше за жінку,1.0


In [33]:
df[df['target']==0].tail(4)

,text,target
813,жінки і хлопчаки дівчата й старі нерухомо сидять одягнені в своє найкраще і дивляться прямо в об'єктив,0.0
814,стереотипи існують і щодо чоловіків як і проблеми спричинені нерівністю,0.0
815,головну героїню фільму талановито зіграла єлизавета максимова,0.0
816,дискримінувати людину через її стать це нонсенс,0.0


In [36]:
ft = fasttext.load_model('cc.uk.300.bin')
ft.get_dimension()

300

In [37]:
X = df['text'].values
y = df['target'].values

In [38]:
def doc_vector(text):
    words = text.split()
    vector = np.mean([ft.get_word_vector(w) for w in words], axis=0)
    return vector

In [39]:
X_vec = [doc_vector(sample) for sample in X]
X_train, X_test, y_train, y_test = train_test_split(X_vec, y.tolist(), test_size=0.3, random_state=42, shuffle=True)

In [49]:
from sklearn.svm import SVC
clf = SVC(kernel='linear').fit(X_train, y_train)

In [50]:
%%time
predicted = clf.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

         0.0       0.84      0.87      0.85       147
         1.0       0.80      0.75      0.77        99

    accuracy                           0.82       246
   macro avg       0.82      0.81      0.81       246
weighted avg       0.82      0.82      0.82       246

CPU times: user 36.6 ms, sys: 392 µs, total: 37 ms
Wall time: 35.9 ms


In [52]:
!tail gold_lower.txt

__label__no_sexism вона стане наймолодшою представницею країни в міжнародних конкурсах краси першого рівня повідомила корреспондент net прес-служба національного комітету королеви світу україна
__label__sexism жінка самотня ось і тормозить
__label__no_sexism я не розумію звідки береться ця чоловіча солідарність коли абсолютно різні я впевнений люди з різним рівнем освіти досвіду географією всі як один встають і починають кричати що це неправда феміністки вигадали і навіть не читаючи стверджують що це повна маячня
__label__sexism жінкам від нас потрібні тільки гроші
__label__sexism жінки самі провокують чоловіків своїм непристойним одягом та поведінкою
__label__sexism на жаль мало людей задають собі питання а ще рідше це роблять жінки а що в мені такого що я зробила не так що спровокувала таку поведінку такого чоловіка
__label__no_sexism ми впевнені що партнерство сприятиме гендерним трансформаціям в україні а також допоможе досягти спільної мети процвітаючої і демократичної україни де 

In [ ]:
!head -n 574 gold_lower.txt > gold.train
!tail -n 244 gold_lower.txt > gold.valid

In [53]:
model = fasttext.train_supervised(input="gold.train", epoch=25, wordNgrams=2)
model.test('gold.valid')

(244, 0.8073770491803278, 0.8073770491803278)

In [54]:
model = fasttext.train_supervised(input="gold.train", pretrainedVectors='cc.uk.300.vec', dim=300, wordNgrams=2)
model.test('gold.valid')

(244, 0.8401639344262295, 0.8401639344262295)

In [ ]:
%%time
corpus_file = 'sents.txt'
initial_model_path = "cc.uk.300.bin"
ft_model = gensim.models.fasttext.load_facebook_model(initial_model_path)
ft_model.train(corpus_file=corpus_file, total_words=len(ft_model.wv.vocab), epochs=25, threads=6)
ft_model.save('gensim_output/fem25.cc.uk.300.bin')
print("Embeddings are successfully trained.")

In [ ]:
ft_model = gensim.models.fasttext.FastText.load("gensim_output/fem25.cc.uk.300.bin")
len(ft_model.wv.vocab)

In [ ]:
with open('vocab.vec', 'w') as f:
    print(len(ft_model.wv.vocab),300,file=f)
    for word in ft_model.wv.vocab.keys():
        print(word, ' '.join(f'{i:.04f}' for i in ft_model.wv[word]), file=f)

In [3]:
model = fasttext.train_supervised(input="gold.train", pretrainedVectors='vocab.vec', dim=300, wordNgrams=2)
model.test('gold.valid')

(244, 0.8524590163934426, 0.8524590163934426)

In [4]:
model.save_model("final.bin")